In [7]:
import math
import string

tests = """
3:20
"""

class Range:
    def __init__(self, start, stop, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        
    def evaluate(self):
        return list(range(self.start, self.stop+1, self.step))

class Program:
    def __init__(self, source):
        self.source = source
        self.tree = []
        self.parse()
        
    def parse(self):
        statements = self.source.replace('|', '\n').split('\n')
        for s in statements:
            statement_parse = []
                
        
M = Program(tests)